In [1]:
from model import ResNet50, BERT_Module, TripletNetwork, TextReprojection, ImageTextRetrievalModel
from data import ImageTextRetrievalDataset, ImageTextDataModule
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.nn import TripletMarginLoss
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
from utils import get_image_encoder_output_size, get_text_encoder_output_size
from transforms import albumentations_transform

/home/georg/projects/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = ImageTextRetrievalModel(text_embedding_dim=768, 
                                image_embedding_dim=2048, 
                                learning_rate=1e-4,
                                mode='text2img'
                                )

/home/georg/projects/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/georg/projects/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:

lr_scheduler = CosineAnnealingLR(model.optimizer, T_max=10, eta_min=0.0001, last_epoch=-1)
model.lr_scheduler = lr_scheduler

In [8]:
dm = ImageTextDataModule(batch_size=32, num_workers=4, transform=albumentations_transform())

In [9]:
for batch in dm.train_dataloader():
    print(len(batch[0]), len(batch[1]))
    break

32 32


In [10]:
trainer = Trainer(max_epochs=10, 
                  accelerator='auto', 
                  #callbacks=[lr_scheduler],
                  log_every_n_steps=10,)
                  #fast_dev_run=True)

trainer.fit(model, dm)

ValueError: Expected a parent